In [1]:
## PYSPARK INTERVIEW QUESTIONS - ANSH LAMBA

In [2]:
import findspark
findspark.init()

In [3]:
import os
import sys

# Replace this with your actual Python path
python_path = sys.executable  # This is safe — it auto-detects your current Python path

os.environ["PYSPARK_PYTHON"] = python_path
os.environ["PYSPARK_DRIVER_PYTHON"] = python_path

In [4]:
from pyspark.sql.functions import * 
from pyspark.sql.types import *

from pyspark import SparkContext, SparkConf 
from pyspark.conf import SparkConf 
from pyspark.sql import SparkSession, HiveContext,DataFrame
from pyspark.sql.window import Window
from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StringType, StructField, StringType,LongType,DecimalType,DateType,TimestampType, IntegerType,DoubleType

In [5]:
## SparkSession
spark = SparkSession.builder \
                        .appName('example-pyspark-read-and-write-from-hive') \
                        .master("local[*]") \
                        .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0,com.crealytics:spark-excel_2.12:3.3.3_0.20.3") \
                        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
                        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
                        .config("spark.driver.memory", "4g") \
                        .config("spark.executor.memory", "2g") \
                        .config("spark.sql.execution.arrow.pyspark.enabled", "false")\
                        .enableHiveSupport() \
                        .getOrCreate()



In [6]:
sda

NameError: name 'sda' is not defined

In [85]:
### While analyzing customer reviews, you need to identify the most frequently used words in the feedback. How would you implement this?

In [86]:
data = [("customer1", "The product is great"), ("customer2", "Great product, fast delivery"), ("customer3", "Not bad, could be better")]
columns = ["customer_id", "feedback"]

df = spark.createDataFrame(data, columns)

In [87]:
df.show(5,False)

+-----------+----------------------------+
|customer_id|feedback                    |
+-----------+----------------------------+
|customer1  |The product is great        |
|customer2  |Great product, fast delivery|
|customer3  |Not bad, could be better    |
+-----------+----------------------------+



In [88]:
# df = df.withColumn('feedback', split(f.col('feedback'),' '))
# df.show(5,False)

In [89]:
df = df.withColumn('feedback',f.explode(f.split('feedback',' ')))\
        .withColumn('feedback',lower('feedback'))\
        .groupBy('feedback').agg(count('feedback').alias('wordCount'))

In [90]:
df.show()

+--------+---------+
|feedback|wordCount|
+--------+---------+
|   great|        2|
|      is|        1|
|     the|        1|
| product|        1|
|    fast|        1|
|delivery|        1|
|product,|        1|
|   could|        1|
|     not|        1|
|      be|        1|
|    bad,|        1|
|  better|        1|
+--------+---------+



In [91]:
## 9. You need to calculate the cumulative sum of sales over time for each product. How would you approach this?

In [92]:
data = [("product1", "2023-12-01", 100), ("product2", "2023-12-02", 200),
        ("product1", "2023-12-03", 150), ("product2", "2023-12-04", 250)]
columns = ["product_id", "date", "sales"]
df = spark.createDataFrame(data, columns)
df.show()

+----------+----------+-----+
|product_id|      date|sales|
+----------+----------+-----+
|  product1|2023-12-01|  100|
|  product2|2023-12-02|  200|
|  product1|2023-12-03|  150|
|  product2|2023-12-04|  250|
+----------+----------+-----+



In [93]:
## cast date column from string to Datestamp:

df = df.withColumn("date", f.col('date').cast(DateType()))
## applying window function with Sum as aggregation:::


df = df.withColumn("CumalativeSum", sum("sales").over(Window.partitionBy('product_id').orderBy(f.col("date")))) \
        .orderBy(f.col("date"), ascending=True)

#.orderBy(f.col("totalActions"),ascending=False)

In [94]:
df.show()

+----------+----------+-----+-------------+
|product_id|      date|sales|CumalativeSum|
+----------+----------+-----+-------------+
|  product1|2023-12-01|  100|          100|
|  product2|2023-12-02|  200|          200|
|  product1|2023-12-03|  150|          250|
|  product2|2023-12-04|  250|          450|
+----------+----------+-----+-------------+



In [95]:
####################
### 10. While preparing a data pipeline, you notice some duplicate rows in a dataset. How would you remove the duplicates without affecting 
## the original order?

In [96]:
data = [("John", 25), ("Jane", 30), ("John", 25), ("Alice", 22)]
columns = ["name", "age"]
df = spark.createDataFrame(data, columns)
df.show()

+-----+---+
| name|age|
+-----+---+
| John| 25|
| Jane| 30|
| John| 25|
|Alice| 22|
+-----+---+



In [97]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)



In [98]:
## Resolving it with rownumber() window function, when we are getting 2 just filter it out

df = df.withColumn('Flag', row_number().over(Window.partitionBy("name").orderBy(f.col("age").asc()))) \
        .filter(f.col("flag") == 1)\
        .select("name", "age")
        
df.show()

+-----+---+
| name|age|
+-----+---+
|Alice| 22|
| Jane| 30|
| John| 25|
+-----+---+



In [99]:
## 11. You are working with user activity data and need to calculate the average session duration per user. How would you implement this?

In [100]:
data = [("user1", "2023-12-01", 50), ("user1", "2023-12-02", 60), 
        ("user2", "2023-12-01", 45), ("user2", "2023-12-03", 75)]
columns = ["user_id", "session_date", "duration"]
df = spark.createDataFrame(data, columns)

df.show()

+-------+------------+--------+
|user_id|session_date|duration|
+-------+------------+--------+
|  user1|  2023-12-01|      50|
|  user1|  2023-12-02|      60|
|  user2|  2023-12-01|      45|
|  user2|  2023-12-03|      75|
+-------+------------+--------+



In [101]:
## avg:::

df=df.groupBy('user_id').agg(avg('duration').alias("avgDuaration"))
df.show()

+-------+------------+
|user_id|avgDuaration|
+-------+------------+
|  user1|        55.0|
|  user2|        60.0|
+-------+------------+



In [102]:
### 12. While analyzing sales data, you need to find the product with the highest sales for each month. How would you accomplish this?

In [103]:
data = [("product1", "2023-12-01", 100), ("product2", "2023-12-01", 150), 
        ("product1", "2023-12-02", 200), ("product2", "2023-12-02", 250)]
columns = ["product_id", "date", "sales"]
df = spark.createDataFrame(data, columns)
#df.show()

In [104]:
df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- sales: long (nullable = true)



In [105]:
df = df.withColumn("date", f.col("date").cast(DateType()))\
        .withColumn("month",month("date").alias('month'))

## Best Product FOr each Month --> anserw will be product2
## select product_id, month, sum(sales) from table group by product_id, month order by sum(sales) desc;
## ok great, this will be working like that exactly, then we also need to add window function (any ranking function)


In [106]:
df.show()

+----------+----------+-----+-----+
|product_id|      date|sales|month|
+----------+----------+-----+-----+
|  product1|2023-12-01|  100|   12|
|  product2|2023-12-01|  150|   12|
|  product1|2023-12-02|  200|   12|
|  product2|2023-12-02|  250|   12|
+----------+----------+-----+-----+



In [107]:
df = df.groupBy(f.col("product_id"),f.col("month")).agg(sum('sales').alias("sumSales"))\
        .orderBy(f.col('sumSales'), ascending = False)

In [108]:
df.show()

+----------+-----+--------+
|product_id|month|sumSales|
+----------+-----+--------+
|  product2|   12|     400|
|  product1|   12|     300|
+----------+-----+--------+



In [109]:
## Now we will be aplying window function here Desnse rank   ###.desc with windowFunction --->>> alwaysss .....

df = df.withColumn("ranking",dense_rank().over(Window.partitionBy('month').orderBy(f.col("sumSales").desc())))\
        .filter(f.col("ranking") == 1)\
        .select("product_id","month","sumSales")
    

In [110]:
df.show()

+----------+-----+--------+
|product_id|month|sumSales|
+----------+-----+--------+
|  product2|   12|     400|
+----------+-----+--------+



In [111]:
### 13. You are working with a large Delta table that is frequently updated by multiple users. The data is stored in partitions, 
### and sometimes updates can cause inconsistent reads due to concurrent transactions. How would you ensure ACID compliance and 
### avoid data corruption in PySpark?

In [112]:
## ACID properties can be assured using delta log, which is being created predefined when you create a delta table
### For Data corruption we will be using following upserting syntax

In [113]:
## Reading new data which is in parqut format
df = spark.read.format('parquet').load('path')

from delta.tables import DeltaTable

## Reading data from delta table::

deltaTbl = DeltaTable.forPath(spark, "/path/to/delta/table")

deltaTbl.alias('trg').merge(df.alias('src'),"src.id == trg.id")\
                    .whenNotMatchedInsertAll()\
                    .whenMatchedUpdateAll()\
                    .execute()


AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/D:/jupyter_notebooks/path.

In [114]:
## 14. You need to process a large dataset stored in PARQUET format and ensure that all columns have the right schema (Almost). How would you do this?

In [ ]:
df = spark.read.format('parquet')\
                .opion("inferSchema", True)\  ## this will infer schema and you not need to get it manually
                .load("path")

In [ ]:
### 15. You are reading a CSV file and need to handle corrupt records gracefully by skipping them. How would you configure this in PySpark?

df = spark.read.format('csv')\
                .opion("mode", "DROPMALFORMED")\  ## this will drop all malformed records
                .load("path")

In [ ]:
### You have a dataset containing the names of employees and their departments. You need to find the department with the most employees.


In [115]:
data = [("Alice", "HR"), ("Bob", "Finance"), ("Charlie", "HR"), ("David", "Engineering"), ("Eve", "Finance")]
columns = ["employee_name", "department"]

df = spark.createDataFrame(data, columns)
df.show()

+-------------+-----------+
|employee_name| department|
+-------------+-----------+
|        Alice|         HR|
|          Bob|    Finance|
|      Charlie|         HR|
|        David|Engineering|
|          Eve|    Finance|
+-------------+-----------+



In [116]:
df = df.groupBy("department").agg(count(f.col("employee_name")).alias("CountEmployees"))\
        .orderBy(f.col("CountEmployees"),ascending=False)

In [117]:
df.show()

+-----------+--------------+
| department|CountEmployees|
+-----------+--------------+
|         HR|             2|
|    Finance|             2|
|Engineering|             1|
+-----------+--------------+



In [ ]:
### 23. While processing sales data, you need to classify each transaction as either 'High' or 'Low' based on its amount. How would you 
### achieve this using a when condition

In [23]:
data = [("product1", 100), ("product2", 300), ("product3", 50)]
columns = ["product_id", "sales"]

df = spark.createDataFrame(data, columns)
df.show()

+----------+-----+
|product_id|sales|
+----------+-----+
|  product1|  100|
|  product2|  300|
|  product3|   50|
+----------+-----+



In [25]:
df = df.withColumn("classify", when(f.col("sales") > 250,"High").otherwise("Low") )

In [27]:
df.show()

+----------+-----+--------+
|product_id|sales|classify|
+----------+-----+--------+
|  product1|  100|     Low|
|  product2|  300|    High|
|  product3|   50|     Low|
+----------+-----+--------+



In [28]:
## 24. While analyzing a large dataset, you need to create a new column that holds a timestamp of when the record was processed. 
## How would you implement this and what can be the best USE CASE?

In [29]:
data = [("product1", 100), ("product2", 200), ("product3", 300)]
columns = ["product_id", "sales"]

df = spark.createDataFrame(data, columns)
df.show()

+----------+-----+
|product_id|sales|
+----------+-----+
|  product1|  100|
|  product2|  200|
|  product3|  300|
+----------+-----+



In [31]:
import time
processedAt = time.strftime("%Y-%m-%d %H:%M:%S")
df = df.withColumn("processed_at",lit(processedAt))

In [32]:
df.show()

+----------+-----+-------------------+
|product_id|sales|       processed_at|
+----------+-----+-------------------+
|  product1|  100|2025-04-10 14:30:51|
|  product2|  200|2025-04-10 14:30:51|
|  product3|  300|2025-04-10 14:30:51|
+----------+-----+-------------------+



In [35]:
df.collect()

[Row(product_id='product1', sales=100, processed_at='2025-04-10 14:30:51'),
 Row(product_id='product2', sales=200, processed_at='2025-04-10 14:30:51'),
 Row(product_id='product3', sales=300, processed_at='2025-04-10 14:30:51')]

In [ ]:
### 31. Your company uses a large-scale data pipeline that reads from Delta tables and processes data using complex aggregations. 
### However, performance is becoming an issue due to the growing dataset size. How would you optimize the performance of the pipeline?

In [ ]:
%sql

OPTIMIZE tabledelta ZORDER BY ('order_date')

In [ ]:
# Q. what is OPTIMIZE & ZORDER doing here ?

# It (OPTIMIZE) will coalesce the partitions, so it will create fewer partitions of bigger size
# It (ZORDER) will sort the data and inseret it in partitions like buketing do, so in that case you dont need to 
# go to other partitions and data shuffling will be avoided, and if you are not going to second partition than we called as data skiping
  

In [36]:
## 43. You are processing sales data. Group by product categories and create a list of 
### all product names in each category.

In [37]:
data = [("Electronics", "Laptop"), ("Electronics", "Smartphone"), ("Furniture", "Chair"), ("Furniture", "Table")]
columns = ["category", "product"]
df = spark.createDataFrame(data, columns)
df.show()

+-----------+----------+
|   category|   product|
+-----------+----------+
|Electronics|    Laptop|
|Electronics|Smartphone|
|  Furniture|     Chair|
|  Furniture|     Table|
+-----------+----------+



In [38]:
df = df.groupBy("category").agg(collect_list(f.col("product")).alias("products"))

In [39]:
df.show()

+-----------+--------------------+
|   category|            products|
+-----------+--------------------+
|Electronics|[Laptop, Smartphone]|
|  Furniture|      [Chair, Table]|
+-----------+--------------------+



In [ ]:
###  You are analyzing orders. Group by customer IDs and list all unique product IDs each customer purchased.

In [40]:
data = [(101, "P001"), (101, "P002"), (102, "P001"), (101, "P001")]
columns = ["customer_id", "product_id"]
df = spark.createDataFrame(data, columns)
df.show()

+-----------+----------+
|customer_id|product_id|
+-----------+----------+
|        101|      P001|
|        101|      P002|
|        102|      P001|
|        101|      P001|
+-----------+----------+



In [42]:
df = df.groupBy("customer_id").agg(collect_set(f.col("product_id")).alias("product_id"))

In [43]:
df.show()

+-----------+------------+
|customer_id|  product_id|
+-----------+------------+
|        101|[P002, P001]|
|        102|      [P001]|
+-----------+------------+



In [ ]:
### 45. For customer records, combine first and last names only if the email address exists.

In [56]:
data = [("John", "Doe", "john.doe@example.com"), ("Jane", "Smith", None)]
columns = ["first_name", "last_name", "email"]
df = spark.createDataFrame(data, columns)
df.show()

+----------+---------+--------------------+
|first_name|last_name|               email|
+----------+---------+--------------------+
|      John|      Doe|john.doe@example.com|
|      Jane|    Smith|                NULL|
+----------+---------+--------------------+



In [61]:
df = df.withColumn("CombinedName", when(f.col("email").isNotNull(), f.concat(f.col("first_name"), f.col("last_name") ) ).otherwise(None) )

In [62]:
df.show()

+----------+---------+--------------------+------------+
|first_name|last_name|               email|CombinedName|
+----------+---------+--------------------+------------+
|      John|      Doe|john.doe@example.com|     JohnDoe|
|      Jane|    Smith|                NULL|        NULL|
+----------+---------+--------------------+------------+



In [63]:
### OR ###

df = df.withColumn("CombinedName", when(f.col("email").isNotNull(), f.concat_ws("-",f.col("first_name"),f.col("last_name")) ).otherwise(None) )

In [64]:
df.show()

+----------+---------+--------------------+------------+
|first_name|last_name|               email|CombinedName|
+----------+---------+--------------------+------------+
|      John|      Doe|john.doe@example.com|    John-Doe|
|      Jane|    Smith|                NULL|        NULL|
+----------+---------+--------------------+------------+



In [ ]:
## 46. You have a DataFrame containing customer IDs and a list of their purchased product IDs. 
## Calculate the number of products each customer has purchased.

In [65]:
data = [
    (1, ["prod1", "prod2", "prod3"]),
    (2, ["prod4"]),
    (3, ["prod5", "prod6"]),
]
myschema = "customer_id INT ,product_ids array<STRING>"

df = spark.createDataFrame(data, myschema)
df.show()

+-----------+--------------------+
|customer_id|         product_ids|
+-----------+--------------------+
|          1|[prod1, prod2, pr...|
|          2|             [prod4]|
|          3|      [prod5, prod6]|
+-----------+--------------------+



In [66]:
df =df.withColumn("NumberOfProds",size(f.col("product_ids")))
df.show()

+-----------+--------------------+-------------+
|customer_id|         product_ids|NumberOfProds|
+-----------+--------------------+-------------+
|          1|[prod1, prod2, pr...|            3|
|          2|             [prod4]|            1|
|          3|      [prod5, prod6]|            2|
+-----------+--------------------+-------------+



In [ ]:
#You have employee IDs of varying lengths. Ensure all IDs are 6 characters long by padding with leading zeroes.


In [67]:
data = [
    ("1",),
    ("123",),
    ("4567",),
]
schema = ["employee_id"]

df = spark.createDataFrame(data, schema)
df.show()

+-----------+
|employee_id|
+-----------+
|          1|
|        123|
|       4567|
+-----------+



In [71]:
df =df.withColumn("employee_id",f.lpad(f.col("employee_id"),6,"0"))
df.show()

+-----------+
|employee_id|
+-----------+
|     000001|
|     000123|
|     004567|
+-----------+



In [ ]:

# 48. You need to validate phone numbers by checking if they start with "91"

In [72]:
data = [
    ("911234567890",),
    ("811234567890",),
    ("912345678901",),
]
schema = ["phone_number"]

df = spark.createDataFrame(data, schema)
df.show()

+------------+
|phone_number|
+------------+
|911234567890|
|811234567890|
|912345678901|
+------------+



In [73]:
df = df.filter(f.substring(f.col("phone_number"),1,2) == "91")
df.show()

+------------+
|phone_number|
+------------+
|911234567890|
|912345678901|
+------------+



In [74]:
## **49. You have a dataset with courses taken by students. Calculate the average number of courses per student.**

In [75]:
data = [
    (1, ["Math", "Science"]),
    (2, ["History"]),
    (3, ["Art", "PE", "Biology"]),
]
schema = ["student_id", "courses"]

df = spark.createDataFrame(data, schema)
df.show()

+----------+------------------+
|student_id|           courses|
+----------+------------------+
|         1|   [Math, Science]|
|         2|         [History]|
|         3|[Art, PE, Biology]|
+----------+------------------+



In [77]:
df = df.withColumn("CourseTakenCountAvg",f.size(f.col("courses"))).groupBy().agg(avg("CourseTakenCountAvg"))
df.show()

+------------------------+
|avg(CourseTakenCountAvg)|
+------------------------+
|                     2.0|
+------------------------+



In [ ]:
## coalesce SQL USecase:::::

## 50. You have a dataset with primary and secondary contact numbers. Use the primary number if available; otherwise, use the secondary number.

In [78]:
data = [
    (None, "1234567890"),
    ("9876543210", None),
    ("7894561230", "4567891230"),
]
schema = ["primary_contact", "secondary_contact"]

df = spark.createDataFrame(data, schema)
df.show()

+---------------+-----------------+
|primary_contact|secondary_contact|
+---------------+-----------------+
|           NULL|       1234567890|
|     9876543210|             NULL|
|     7894561230|       4567891230|
+---------------+-----------------+



In [79]:
## This problem is similar to coalesce function in sql, we do have coalesce in spoark as well:::


df = df.withColumn("finalContact",f.coalesce(f.col("primary_contact"),f.col("secondary_contact")))
df.show()

+---------------+-----------------+------------+
|primary_contact|secondary_contact|finalContact|
+---------------+-----------------+------------+
|           NULL|       1234567890|  1234567890|
|     9876543210|             NULL|  9876543210|
|     7894561230|       4567891230|  7894561230|
+---------------+-----------------+------------+



In [ ]:
## You are categorizing product codes based on their lengths. If the length is 5, label it as "Standard"; otherwise, label it as "Custom".

In [80]:
data = [
    ("prod1",),
    ("prd234",),
    ("pr9876",),
]
schema = ["product_code"]

df = spark.createDataFrame(data, schema)
df.show()

+------------+
|product_code|
+------------+
|       prod1|
|      prd234|
|      pr9876|
+------------+



In [83]:
df = df.withColumn("flagProdCode", f.when(f.length(f.col("product_code")) == 5,"Standard" ).otherwise("Custom"))

In [84]:
df.show()

+------------+------------+
|product_code|flagProdCode|
+------------+------------+
|       prod1|    Standard|
|      prd234|      Custom|
|      pr9876|      Custom|
+------------+------------+

